In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz raspagem no site da Câmara dos Deputados para pegar links dos sites de deputados e das fotos dos respectivos e cruza com base do TSE para filtrar apenas os eleitos em 2014 
#

In [2]:
from urllib.request import urlopen

In [3]:
from bs4 import BeautifulSoup

In [4]:
import urllib.request, urllib.parse, urllib.error

In [4]:
import pandas as pd

In [ ]:
# Função para acessar página, baixa código HTML e abrir um objeto BeautifulSoup
def prepara_sopa(url):
    html = urlopen(url)
    sopa = BeautifulSoup(html, "lxml")
    return sopa

# Função para pegar lista de nomes e números no menu
def lista_menu(sopa, selector):
    lista_opcoes = sopa.select(selector)[0].find_all("option")
    lista = []
    for item in lista_opcoes:
        lista.append(item.attrs['value']) 
    return lista

# Função que cria URL de cada deputado
def gera_url(numero):
    link = "http://www.camara.leg.br/internet/Deputado/dep_Detalhe.asp?id=" + numero
    return link

# URL raiz
url = "http://www2.camara.leg.br/deputados/pesquisa"
sopa = prepara_sopa(url)

# Seletor do menu
seletor_deputados = "#deputado"
nomes_num = lista_menu(sopa, seletor_deputados)

# Iteração para pegar dados dos deputados e limpar informações
lista_deputados = []
for linha in nomes_num:
    tamanho = len(linha)
    if tamanho > 0:
        localiza1 = linha.find('?')
        exato1 = (localiza1-tamanho)+1
        numero = linha[exato1:]
        #print(numero)
        localiza2 = linha.find('|')
        exato2 = (localiza2-tamanho)
        nome = linha[:exato2]
        #print(nome)
        endereco = gera_url(numero)
        #print(endereco)
        # extrai link da foto
        sopa = prepara_sopa(endereco)
        foto = sopa.find('img', {'class': 'image-left'}).get('src')
        #print(foto)
        lista_deputados.append([nome, numero, endereco, foto])

# Cria dataframe
deputados_sites = pd.DataFrame(lista_deputados, columns = ['Nome', 'Num_referencia', 'Link', 'Link_foto'])

In [7]:
deputados_sites.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 4 columns):
Nome              512 non-null object
Num_referencia    512 non-null object
Link              512 non-null object
Link_foto         512 non-null object
dtypes: object(4)
memory usage: 16.1+ KB


In [9]:
deputados_sites.head().reset_index()

,index,Nome,Num_referencia,Link,Link_foto
0,0,ABEL MESQUITA JR.,178957,http://www.camara.leg.br/internet/Deputado/dep...,http://www.camara.gov.br/internet/deputado/ban...
1,1,ADAIL CARNEIRO,178864,http://www.camara.leg.br/internet/Deputado/dep...,http://www.camara.gov.br/internet/deputado/ban...
2,2,ADALBERTO CAVALCANTI,178914,http://www.camara.leg.br/internet/Deputado/dep...,http://www.camara.gov.br/internet/deputado/ban...
3,3,ADELMO CARNEIRO LEÃO,178890,http://www.camara.leg.br/internet/Deputado/dep...,http://www.camara.gov.br/internet/deputado/ban...
4,4,ADELSON BARRETO,178968,http://www.camara.leg.br/internet/Deputado/dep...,http://www.camara.gov.br/internet/deputado/ban...


In [11]:
# Cruza com TSE para ter apenas deputados eleitos

In [12]:
candidatos = pd.read_csv("candidatos_nome_urna_csv.csv",sep=';',encoding = 'latin_1')

In [13]:
 eleitos = candidatos[(candidatos['Situacao'] == 'ELEITO POR QP') | (candidatos['Situacao'] == 'ELEITO POR MÉDIA') | (candidatos['Situacao'] == 'ELEITO')]

In [14]:
eleitos_d = eleitos[(eleitos['Cargo'] == 'DEPUTADO FEDERAL')]

In [15]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 23 to 26219
Data columns (total 7 columns):
Estado            513 non-null object
Cargo             513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null int64
Nome_urna         513 non-null object
Partido           513 non-null object
Situacao          513 non-null object
dtypes: int64(1), object(6)
memory usage: 32.1+ KB


In [ ]:
# Comparação pelo campo Nome urna da base do TSE, que deveria corresponder com o nome no site da Câmara

In [ ]:
# É a comparação possível porque a base raspada no site não tem o CPF

In [16]:
sites_eleitos = pd.merge(deputados_sites, eleitos_d, left_on='Nome', right_on='Nome_urna')

In [17]:
sites_eleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 340
Data columns (total 11 columns):
Nome              341 non-null object
Num_referencia    341 non-null object
Link              341 non-null object
Link_foto         341 non-null object
Estado            341 non-null object
Cargo             341 non-null object
Nome_candidato    341 non-null object
CPF               341 non-null int64
Nome_urna         341 non-null object
Partido           341 non-null object
Situacao          341 non-null object
dtypes: int64(1), object(10)
memory usage: 32.0+ KB


In [18]:
sites_eleitos.to_csv('dia21_11_eleitos.csv')

In [1]:
# O cruzamento mostrou 171 nomes faltando - podem ser nomes com nome urna diferente ou suplentes

In [20]:
# outer: usa união de chaves de ambos os quadros, semelhante a uma junção externa completa do SQL; classificar chaves lexicograficamente
# indicator: Se True, adiciona uma coluna para saída DataFrame chamado "_merge" com informações sobre a origem de cada linha. Se uma seqüência de caracteres, a coluna com informações sobre a fonte de cada linha será adicionada para saída DataFrame, e a coluna será denominada valor da string. A coluna de informação é de tipo Categorical e assume um valor de "left_only" para observações cuja chave de mesclagem só aparece no DataFrame 'esquerdo', "right_only" para observações cuja chave de mesclagem aparece apenas no DataFrame 'direito' e "ambos" se o A chave de mesclagem da observação é encontrada em ambos.

In [21]:
sites_naoeleitos = pd.merge(deputados_sites, 
                        eleitos_d, 
                        left_on='Nome', 
                        right_on='Nome_urna',
                        how='outer',
                        indicator=True)

In [22]:
sites_naoeleitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 684 entries, 0 to 683
Data columns (total 12 columns):
Nome              512 non-null object
Num_referencia    512 non-null object
Link              512 non-null object
Link_foto         512 non-null object
Estado            513 non-null object
Cargo             513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null float64
Nome_urna         513 non-null object
Partido           513 non-null object
Situacao          513 non-null object
_merge            684 non-null category
dtypes: category(1), float64(1), object(10)
memory usage: 64.9+ KB


In [20]:
# Agrupa apenas pelo lado direito - a lista do TSE

In [49]:
rdf = sites_naoeleitos.query("_merge == 'right_only'").drop('_merge',axis=1)

In [50]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171 entries, 512 to 682
Data columns (total 11 columns):
Nome              0 non-null object
Num_referencia    0 non-null object
Link              0 non-null object
Link_foto         0 non-null object
Estado            171 non-null object
Cargo             171 non-null object
Nome_candidato    171 non-null object
CPF               171 non-null float64
Nome_urna         171 non-null object
Partido           171 non-null object
Situacao          171 non-null object
dtypes: float64(1), object(10)
memory usage: 16.0+ KB


In [51]:
rdf.to_csv('dia21_11_nao_encontrados.csv')

In [ ]:
# Nesse ponto fiz uma checagem manual dos 171 nomes, verificando quem não era deputado eleito e atualizando os nomes urna

In [2]:
# Depois uni os dois arquivos em um

In [5]:
politicos_perfis = pd.read_csv("dia21_11_politicos.csv",sep=',',encoding = 'utf_8')

In [1]:
# Função para baixar fotos e salvar com nome dos deputados

In [13]:
def baixa_foto(nome, url):
	urllib.request.urlretrieve(url, nome)

for linha, row in politicos_perfis.iterrows():
	nome = (row['Nome_urna'])
	url = (row['Link_foto'])
	baixa_foto(nome, url)	